In [66]:
import pandas as pd
from backorder.utils import load_object
from imblearn.combine import SMOTETomek

In [32]:
df=pd.read_csv('/config/workspace/artifacts/15_02_23__06_24_25/data_ingestion/DATASET/test.csv')

In [4]:
categorical = load_object('/config/workspace/saved_models/0/categorical_encoder/categorical.pkl')
model = load_object('/config/workspace/saved_models/0/model/model.pkl')
target = load_object('/config/workspace/saved_models/0/target_encoder/target.pkl')
numerical = load_object('/config/workspace/saved_models/0/transformer/transformer.pkl')

In [9]:
input_numerical_features = list(numerical.feature_names_in_)
input_categorical_features  = list(categorical.feature_names_in_)

In [33]:
smt = SMOTETomek(random_state=42,sampling_strategy='minority')
df[input_numerical_features] = numerical.transform(df[input_numerical_features])
df[input_categorical_features] = categorical.transform(df[input_categorical_features])
target_df = target.transform(df['went_on_backorder'])

In [35]:
df

,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,sales_9_month,...,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop,went_on_backorder
0,0.504132,0.0,0.0,0.0000,0.00,-0.015385,0.1,0.742857,0.436620,0.411215,...,0.0,0.416667,0.434783,0.0,0.0,0.0,0.0,1.0,0.0,No
1,1.289256,1.0,0.0,0.0000,0.00,-0.015385,0.0,-0.114286,-0.112676,-0.121495,...,0.0,-0.583333,-0.434783,0.0,0.0,0.0,1.0,1.0,0.0,No
2,0.396694,0.0,100.0,0.0000,0.00,-0.015385,-0.1,-0.142857,-0.140845,-0.140187,...,0.0,0.208333,0.434783,0.0,0.0,0.0,0.0,1.0,0.0,No
3,0.041322,0.5,0.0,0.0000,0.00,-0.015385,-0.1,-0.142857,-0.140845,-0.140187,...,0.0,-0.208333,-0.347826,0.0,0.0,0.0,0.0,1.0,0.0,No
4,11.446281,0.0,0.0,0.0000,0.00,-0.015385,0.1,-0.028571,-0.070423,-0.074766,...,0.0,-0.333333,-0.869565,0.0,1.0,0.0,0.0,1.0,0.0,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118286,7.561983,0.0,0.0,0.0000,0.00,-0.015385,0.5,0.800000,0.591549,0.803738,...,0.0,0.333333,0.347826,0.0,0.0,0.0,0.0,1.0,0.0,No
118287,-0.190083,0.0,0.0,0.3125,0.25,0.215385,0.1,0.085714,0.000000,-0.037383,...,0.0,-2.875000,-2.956522,0.0,0.0,0.0,1.0,1.0,0.0,No
118288,-0.066116,0.0,0.0,0.0000,0.00,-0.015385,-0.1,-0.142857,-0.126761,-0.121495,...,0.0,-0.500000,0.173913,0.0,0.0,0.0,0.0,1.0,0.0,No
118289,0.628099,-2.0,4.0,0.0000,0.70,1.061538,1.2,1.057143,1.211268,1.392523,...,0.0,-3.625000,-3.739130,0.0,0.0,0.0,0.0,0.0,0.0,No


In [36]:
input,target = smt.fit_resample(df[input_numerical_features+input_categorical_features].to_numpy(),target_df)

In [38]:
pd.DataFrame(input)

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,0.504132,0.000000,0.0,0.000000,0.000000,-0.015385,0.100000,0.742857,0.436620,0.411215,...,0.0,0.416667,0.434783,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1.289256,1.000000,0.0,0.000000,0.000000,-0.015385,0.000000,-0.114286,-0.112676,-0.121495,...,0.0,-0.583333,-0.434783,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,0.396694,0.000000,100.0,0.000000,0.000000,-0.015385,-0.100000,-0.142857,-0.140845,-0.140187,...,0.0,0.208333,0.434783,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.041322,0.500000,0.0,0.000000,0.000000,-0.015385,-0.100000,-0.142857,-0.140845,-0.140187,...,0.0,-0.208333,-0.347826,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,11.446281,0.000000,0.0,0.000000,0.000000,-0.015385,0.100000,-0.028571,-0.070423,-0.074766,...,0.0,-0.333333,-0.869565,0.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234219,0.350519,1.000000,0.0,0.000000,0.000000,-0.015385,3.756235,1.154809,0.529047,0.758510,...,0.0,-0.833564,-0.521859,0.0,0.0,0.0,0.0,0.0,1.0,0.0
234220,-0.119257,-1.402236,0.0,0.146646,1.030734,0.655011,0.060894,0.142857,0.147613,0.132095,...,0.0,0.449255,0.495263,0.0,0.0,0.0,0.0,1.0,1.0,0.0
234221,-0.231405,0.000000,0.0,0.250000,0.143183,0.103497,0.024244,0.014718,-0.052923,-0.051826,...,0.0,0.301768,0.457179,0.0,0.0,0.0,0.0,0.0,1.0,0.0
234222,-0.223306,-1.500000,0.0,0.000000,0.012250,-0.000308,-0.100000,-0.113713,-0.091126,-0.093271,...,0.0,0.500000,0.522610,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [39]:
y_pred=model.predict(input)

In [29]:
from sklearn.metrics import classification_report,f1_score

In [40]:
print(classification_report(target,y_pred))

              precision    recall  f1-score   support

           0       0.76      1.00      0.86    117112
           1       1.00      0.68      0.81    117112

    accuracy                           0.84    234224
   macro avg       0.88      0.84      0.84    234224
weighted avg       0.88      0.84      0.84    234224



In [41]:
f1_score(target,y_pred)

0.8090680867249904

In [42]:
import numpy as np
np.c_[input,target]

array([[  0.50413223,   0.        ,   0.        , ...,   1.        ,
          0.        ,   0.        ],
       [  1.2892562 ,   1.        ,   0.        , ...,   1.        ,
          0.        ,   0.        ],
       [  0.39669421,   0.        , 100.        , ...,   1.        ,
          0.        ,   0.        ],
       ...,
       [ -0.23140496,   0.        ,   0.        , ...,   1.        ,
          0.        ,   1.        ],
       [ -0.22330603,  -1.5       ,   0.        , ...,   1.        ,
          0.        ,   1.        ],
       [ -0.16762207,  -1.5       ,   0.        , ...,   1.        ,
          0.        ,   1.        ]])

In [43]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(target,y_pred))

[[116813    299]
 [ 37348  79764]]


In [44]:
tn, fp, fn, tp = confusion_matrix(target,y_pred).ravel()

In [45]:
print(f"tp = {tp} fp = {fp}")
print(f"fn = {fn} tn = {tn}")

tp = 79764 fp = 299
fn = 37348 tn = 116813


In [46]:
from sklearn.metrics import roc_auc_score
roc_auc_score(target,y_pred)

0.8392692465332331

In [ ]:
## Hyper parameter tuning

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

In [80]:
import pandas as pd
from backorder.utils import load_object
from imblearn.ensemble import BalancedRandomForestClassifier

In [81]:
df=pd.read_csv('/config/workspace/artifacts/15_02_23__06_24_25/data_ingestion/DATASET/train.csv')

In [93]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.drop('went_on_backorder',axis=1),df['went_on_backorder'],random_state=42,test_size=0.33)

In [94]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler
numerical = Pipeline(steps=[
            ('si',SimpleImputer(strategy='median')),
            ('rs',RobustScaler()),
            ])

In [97]:

X_train[input_numerical_features] = numerical.fit_transform(X_train[input_numerical_features])
X_test[input_numerical_features] = numerical.transform(X_test[input_numerical_features])


In [99]:

from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(drop='if_binary',sparse_output=False)
X_train[input_categorical_features] = ohe.fit_transform(X_train[input_categorical_features])
X_test[input_categorical_features] = ohe.transform(X_test[input_categorical_features])

In [100]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

In [108]:
clf = BalancedRandomForestClassifier(random_state=42,sampling_strategy='not majority')

In [109]:
clf.fit(X_train,y_train)

/config/.local/lib/python3.8/site-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function delayed is deprecated; The function `delayed` has been moved from `sklearn.utils.fixes` to `sklearn.utils.parallel`. This import path will be removed in 1.5.
  warnings.warn(msg, category=FutureWarning)
/config/.local/lib/python3.8/site-packages/sklearn/utils/parallel.py:114: UserWarning: `sklearn.utils.parallel.delayed` should be used with `sklearn.utils.parallel.Parallel` to make it possible to propagate the scikit-learn configuration of the current thread to the joblib workers.
  warnings.warn(
/config/.local/lib/python3.8/site-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function delayed is deprecated; The function `delayed` has been moved from `sklearn.utils.fixes` to `sklearn.utils.parallel`. This import path will be removed in 1.5.
  warnings.warn(msg, category=FutureWarning)
/config/.local/lib/python3.8/site-packages/sklearn/utils/parallel.py:114: UserWarning: `sklear

BalancedRandomForestClassifier(random_state=42,
                               sampling_strategy='not majority')

In [54]:
pd.DataFrame(input)

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,0.504132,0.000000,0.0,0.000000,0.000000,-0.015385,0.100000,0.742857,0.436620,0.411215,...,0.0,0.416667,0.434783,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1.289256,1.000000,0.0,0.000000,0.000000,-0.015385,0.000000,-0.114286,-0.112676,-0.121495,...,0.0,-0.583333,-0.434783,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,0.396694,0.000000,100.0,0.000000,0.000000,-0.015385,-0.100000,-0.142857,-0.140845,-0.140187,...,0.0,0.208333,0.434783,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.041322,0.500000,0.0,0.000000,0.000000,-0.015385,-0.100000,-0.142857,-0.140845,-0.140187,...,0.0,-0.208333,-0.347826,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,11.446281,0.000000,0.0,0.000000,0.000000,-0.015385,0.100000,-0.028571,-0.070423,-0.074766,...,0.0,-0.333333,-0.869565,0.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234261,0.350519,1.000000,0.0,0.000000,0.000000,-0.015385,3.756235,1.154809,0.529047,0.758510,...,0.0,-0.833564,-0.521859,0.0,0.0,0.0,0.0,0.0,1.0,0.0
234262,-0.119257,-1.402236,0.0,0.146646,1.030734,0.655011,0.060894,0.142857,0.147613,0.132095,...,0.0,0.449255,0.495263,0.0,0.0,0.0,0.0,1.0,1.0,0.0
234263,-0.231405,0.000000,0.0,0.250000,0.143183,0.103497,0.024244,0.014718,-0.052923,-0.051826,...,0.0,0.301768,0.457179,0.0,0.0,0.0,0.0,0.0,1.0,0.0
234264,-0.223306,-1.500000,0.0,0.000000,0.012250,-0.000308,-0.100000,-0.113713,-0.091126,-0.093271,...,0.0,0.500000,0.522610,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [110]:
train_pred = clf.predict(X_train)
test_pred = clf.predict(X_test)

In [111]:
from sklearn.metrics import classification_report,f1_score

In [112]:
print(classification_report(y_train,train_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    159499
           1       1.00      0.98      0.99      1410

    accuracy                           1.00    160909
   macro avg       1.00      0.99      1.00    160909
weighted avg       1.00      1.00      1.00    160909



In [113]:
print(classification_report(y_test,test_pred))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00     78527
           1       0.65      0.06      0.10       728

    accuracy                           0.99     79255
   macro avg       0.82      0.53      0.55     79255
weighted avg       0.99      0.99      0.99     79255



In [59]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(target,y_pred))

[[116834    299]
 [ 37364  79769]]


In [ ]:
tn, fp, fn, tp = confusion_matrix(target,y_pred).ravel()

In [60]:
print(f"tp = {tp} fp = {fp}")
print(f"fn = {fn} tn = {tn}")

tp = 79764 fp = 299
fn = 37348 tn = 116813


In [61]:
from sklearn.metrics import roc_auc_score
roc_auc_score(target,y_pred)

0.8392297644557897

In [63]:
smt.n_features_in_

21

In [64]:
smt.get_feature_names_out

<bound method OneToOneFeatureMixin.get_feature_names_out of SMOTE(random_state=42, sampling_strategy='minority')>

In [ ]:
smt.features